In [1]:
import requests
import json
from pprint import pprint 
import pandas as pd
from sqlalchemy import create_engine, inspect
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session
import sqlite3
Base = declarative_base()

In [2]:
# Pull in combined data for airlines

American_Alaskan = pd.read_csv("Resources/CSV/Combined/AA_concat.csv")
Delta = pd.read_csv("Resources/CSV/Combined/Delta_Airlines.csv")
Southwest = pd.read_csv("Resources/CSV/Combined/Southwest_Airlines.csv")
United = pd.read_csv("Resources/CSV/Combined/United_Airlines.csv")

# Clean up column headers so that all data is matching
American_Alaskan.rename(columns={'Origin Airport':'Origin_Airport',
                          'Carrier Code':'Carrier_Code',
                          'Destination Airport':'Destination_Airport',
                          'Date (MM/DD/YYYY)':'Date'},
                 inplace=True)

AA = American_Alaskan[American_Alaskan.columns.drop('Unnamed: 0')]

Delta.rename(columns={'Carrier Code':'Carrier_Code',
                      'Date (MM/DD/YYYY)':'Date'},
                 inplace=True)

Southwest.rename(columns={'Date (MM/DD/YYYY)':'Date'},
                inplace=True)

United.rename(columns={'Date (MM/DD/YYYY)':'Date'},
                inplace=True)

In [3]:
# Create SQLite engine
engine = create_engine('sqlite:///flights.sqlite', echo=False)
inspector = inspect(engine)
Base.metadata.create_all(engine)
session= Session(engine)

In [4]:
# Create table and append airline flights data
con=engine
con.execute('DROP TABLE IF EXISTS flights')
AA.to_sql('flights', con)
Delta.to_sql('flights', con, if_exists='append', index=False)
Southwest.to_sql('flights', con, if_exists='append', index=False)
United.to_sql('flights', con, if_exists='append', index=False)

# Count the data stored within the newly created database
engine.execute("SELECT COUNT(*) FROM flights").fetchall()

[(2365604,)]

In [5]:
# Database too large. Drop data from all years except 2018.

engine.execute("DELETE FROM flights WHERE Date LIKE '%2019%'")
engine.execute("DELETE FROM flights WHERE Date LIKE '%2017%'")
engine.execute("DELETE FROM flights WHERE Date LIKE '%2016%'")
engine.execute("DELETE FROM flights WHERE Date LIKE '%2015%'")
engine.execute("DELETE FROM flights WHERE Date LIKE '%2014%'")

engine.execute("SELECT COUNT(*) FROM flights").fetchall()

[(496336,)]

In [6]:
# Use vacuum to rebuild the main data base and change the file size from 217MB to 46,232KB
engine.execute("vacuum")

In [7]:
# Put Flights sqlite database into a data frame and rename column headers that were dropped

flights_db = pd.DataFrame(engine.execute("SELECT * FROM flights").fetchall())

flights_db = flights_db.rename(columns={1:'Carrier_Code', 2:'Date', 
                  5:'Destination_Airport', 10:'Departure_Delay',
                  18:'Origin_Airport'
                          })

flights_db = flights_db.drop(columns={0, 3, 4, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 17})

flights_db

,Carrier_Code,Date,Destination_Airport,Departure_Delay,Origin_Airport
0,American Airlines,01/01/2018,DFW,1.0,ATL
1,American Airlines,01/01/2018,DFW,-2.0,ATL
2,American Airlines,01/01/2018,DFW,-4.0,ATL
3,American Airlines,01/01/2018,DFW,-3.0,ATL
4,American Airlines,01/01/2018,LAX,-4.0,ATL
5,American Airlines,01/01/2018,ORD,1.0,ATL
6,American Airlines,01/01/2018,DFW,14.0,ATL
7,American Airlines,01/01/2018,ORD,-2.0,ATL
8,American Airlines,01/01/2018,DFW,-3.0,ATL
9,American Airlines,01/01/2018,DFW,-3.0,ATL


In [8]:
# jsonify the dataframe

flights_db.to_json(r"Resources/json/flights.json", orient='index')